In [1]:
!pip install gitpython


In [9]:
!pip install --upgrade pip setuptools wheel
!pip install numpy==1.26.4 --only-binary :all:




  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.2
    Uninstalling numpy-1.23.2:
      Successfully uninstalled numpy-1.23.2


In [11]:
!pip install demucs



     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.2 MB 1.6 MB/s eta 0:00:01
     ----------- ---------------------------- 0.3/1.2 MB 4.2 MB/s eta 0:00:01
     --------------------- ------------------ 0.6/1.2 MB 5.8 MB/s eta 0:00:01
     ---------------------------------------- 1.2/1.2 MB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/87.1 kB ? eta -:--:--
     ---------------------------------------- 87.1/87.1 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ----------------------------

In [ ]:
import os
import shutil
from git import Repo
from spleeter.separator import Separator
import argparse

def split_song(input_file, output_folder):
    """Split a song into vocals and instruments using Spleeter, save with custom names."""
    separator = Separator('spleeter:2stems')
    base_name = os.path.splitext(os.path.basename(input_file))[0]
    temp_output_path = os.path.join(output_folder, "temp_" + base_name)
    if not os.path.exists(temp_output_path):
        os.makedirs(temp_output_path)
    separator.separate_to_file(input_file, temp_output_path)
    vocals_path = os.path.join(output_folder, f"{base_name}_vocals.wav")
    instruments_path = os.path.join(output_folder, f"{base_name}_instruments.wav")
    shutil.move(os.path.join(temp_output_path, f"{base_name}/vocals.wav"), vocals_path)
    shutil.move(os.path.join(temp_output_path, f"{base_name}/accompaniment.wav"), instruments_path)
    shutil.rmtree(temp_output_path)
    print(f"Tracks for {base_name} saved as {vocals_path} and {instruments_path}")

def process_repository(repo_url, local_path, output_folder):
    """Clone a GitHub repository, process each song, and output to a specific folder."""
    if not os.path.exists(local_path):
        Repo.clone_from(repo_url, local_path)
        print(f"Repository cloned at {local_path}")
    else:
        print(f"Repository already exists at {local_path}")

    for file_name in os.listdir(local_path):
        if file_name.lower().endswith(('.mp3', '.wav')):
            file_path = os.path.join(local_path, file_name)
            split_song(file_path, output_folder)

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Process all songs in a GitHub repository.")
    parser.add_argument("repo_url", type=str, help="URL of the GitHub repository containing the audio files.")
    parser.add_argument("local_path", type=str, help="Local path to clone the repository.")
    parser.add_argument("output_folder", type=str, help="Folder to save the output tracks.")

    args = parser.parse_args()

    # Ensure the output directory exists
    if not os.path.exists(args.output_folder):
        os.makedirs(args.output_folder)

    # Process the repository
    process_repository(args.repo_url, args.local_path, args.output_folder)
